nonferrous metals repeats, not clear i have the right itemlist period, simplify to 10 major groups or something...maybe do fx outperformance vs peers instead of reer, or a basket of EUR/JPY/GBP/USD FX instead?

In [1]:
from pandas_datareader import data
from ipynb.fs.full.quick_functions import *
from plotly.subplots import make_subplots
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from plotly.subplots import make_subplots

df_acronyms = pd.read_csv("Downloads/BLS_PPI_codes.csv").iloc[:,0:2]
df_acronyms.columns = ["codes","description"]
mydict = dict(zip(df_acronyms["codes"],df_acronyms["description"]))
mydict

#adding WPU15 improves R2 materially but is just "miscellaneous products"

itemlist = ['WPU01',
 'WPU02',
 'WPU03',
 'WPU04',
 'WPU06',
 'WPU07',
 'WPU08',
 'WPU09',
 'WPU10',
 'WPU11',
 'WPU12',
 'WPU13',
 'WPU14',
 'WPU101',
 'WPU102',
 'WPU057',
 'WPU061',
 'WPU071',
 'WPU081']
    
start_date = '2004-01-01'
end_date = '2023-09-23'

df = data.DataReader(itemlist, 'fred', start_date, end_date)
df = df / df.iloc[-1] * 100
df.isnull().sum()

WPU01     0
WPU02     0
WPU03     0
WPU04     0
WPU06     0
WPU07     0
WPU08     0
WPU09     0
WPU10     0
WPU11     0
WPU12     0
WPU13     0
WPU14     0
WPU101    0
WPU102    0
WPU057    0
WPU061    0
WPU071    0
WPU081    0
dtype: int64

In [2]:
df2 = pd.read_excel("https://www.bis.org/statistics/eer/broad.xlsx",header=3)
df2.index= df2.iloc[:, 0]
df_reer = df2.iloc[1:,1:]
df_reer = (df_reer / df_reer.rolling(60).mean() - 1).iloc[60:] * 100
df_reer.head(20)
df_reer.columns

Index(['Algeria', 'Argentina', 'Australia', 'Austria', 'Belgium',
       'Bosnia & Herzegovina', 'Brazil', 'Bulgaria', 'Canada', 'Chile',
       'China', 'Chinese Taipei', 'Colombia', 'Croatia', 'Cyprus', 'Czechia',
       'Denmark', 'Estonia', 'Euro area', 'Finland', 'France', 'Germany',
       'Greece', 'Hong Kong SAR', 'Hungary', 'Iceland', 'India', 'Indonesia',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Korea', 'Latvia', 'Lithuania',
       'Luxembourg', 'Malaysia', 'Malta', 'Mexico', 'Morocco', 'Netherlands',
       'New Zealand', 'North Macedonia', 'Norway', 'Peru', 'Philippines',
       'Poland', 'Portugal', 'Romania', 'Russia', 'Saudi Arabia', 'Serbia',
       'Singapore', 'Slovakia', 'Slovenia', 'South Africa', 'Spain', 'Sweden',
       'Switzerland', 'Thailand', 'Turkey', 'United Arab Emirates',
       'United Kingdom', 'United States'],
      dtype='object')

In [3]:
reer_index = df_reer.index.month.astype(str) + "-" + df_reer.index.year.astype(str)
commod_index = df.index.month.astype(str) + "-" + df.index.year.astype(str)

df_reer.index = reer_index
df.index = commod_index

In [7]:
currency = "Colombia"
years = 19
test_df = df[itemlist].merge(df_reer[currency],left_index=True,right_index=True,how="inner").iloc[-years*12:]
y = test_df[currency]
X = test_df[itemlist]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42)
lasso = Lasso(max_iter=10000)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

test_df["ypred"] = lasso.predict(X)
test_df["resids"] = (test_df[currency] - test_df["ypred"])

coefficients = lasso.coef_
r_squared = r2_score(y_test, lasso.predict(X_test))

X_colnames = X.columns
selected_X_vars = []
for i in range(len(coefficients)):
    if coefficients[i] != 0:
        selected_X_vars.append(X_colnames[i])
non_zero_coefficients = [round(x,3) for x in coefficients if x != 0]
labellist = []
for x in selected_X_vars:
    labellist.append(mydict[x.lower()])

print(currency)
print("Mean squared error:", round(mse,2))
print("R-squared:", round(r_squared,2))
print("Coefficients:", non_zero_coefficients)
print("Relevant Variables:",labellist)

title = f"R^2: {round(r_squared,2)}"
fig = make_subplots(rows=1, cols=2)
fig.add_scatter(x=test_df.index, y=test_df[currency], mode='lines', name=currency, row=1, col=1)
fig.add_scatter(x=test_df.index, y=test_df["ypred"], mode='lines', name="Model",row=1, col=1)
fig.add_scatter(x=test_df.index, y=test_df["resids"], mode='lines', name="Residual (%)", row=1, col=2)
fig.update_layout(width=1000, height=500, title_text=f"{currency} REER vs 5y average and model, {title}")
fig.update_layout(showlegend=True)
fig.show()

Colombia
Mean squared error: 21.28
R-squared: 0.83
Coefficients: [-0.067, -0.409, -0.263, -0.366, -0.995, 0.11, 0.273, 0.308, 0.516, 0.306]
Relevant Variables: ['Farm products', 'Textile products and apparel', 'Hides, skins, leather, and related products', 'Chemicals and allied products', 'Lumber and wood products', 'Iron and steel', 'Nonferrous metals', 'Petroleum products, refined', 'Rubber and rubber products', 'Lumber']


In [5]:
selected_X_vars

['WPU02',
 'WPU04',
 'WPU06',
 'WPU11',
 'WPU102',
 'WPU057',
 'WPU061',
 'WPU071',
 'WPU081']